#### Recommender System
**Course:** Data Mining <br>
**Authors:** Lada Morozova, Shamil Arslanov, Danis Alukaev, Maxim Faleev, Rizvan Iskaliev <br>
**Group:** B19-DS-01

## 0 Prerequisites

In [1]:
import os 
import random
import time
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch as torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
from pathlib import Path

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Step 4. Modeling

## 1. Select Modeling Technique

This section describes modelling tecniques employed in this project. We have organised our development process in iterative manner, i.e. on each iteration we came up with a novel architecture that potentially outperforms the current SoA method. Recent advances in the field were taken from ["A review on deep learning for recommender systems: challenges and remedies"](https://link.springer.com/article/10.1007/s10462-018-9654-y) by Batmaz Z. et al. In total, there are four different implemented models: item popularity model, multilayer perceptron, factorization machine, and behaviour sequence transformer. 

### 1.1 Item Popularity Model

Model grounds on assumption that most popular movies are likely to be interested for a user. We define popularity in terms of ratings number. Thus, for each user recommender system always return top-k movies with a highest number of ratings. Certainly, this is a naive approach and is not likely to satisfy our data mining goal. For this reason, we will assume that the company uses this model in its current setup, and benchmarking of all other (more sophisticated) models will be performed in comparison with performance of this "baseline".

Although the model is simple, the great thing about it is the lack of assumptions about the data. Essentially, we can apply this method to any dataset until it contains user ratings and names of movies.

### 1.2 Multilayer Perceptron

Another quite natural option is to use Multilayer Perceptron (MLP). As an input this model takes concatenated latent representation of user, movie, age, occupation, gender, as well as other manually derived on previous step features. This data is passed through multiple dense layers with non-linear activations, e.g. rectified linear unit. The output layer is a single neuron with logit, that is activated using sigmoid function. The result can be interpreted as a probability that the user will enjoy the movie.

The developed model uses PyTorch embeddings to encode user, movie, age, occupation and gender. Therefore, the input tensor should consist of indices that can be refered to these embeddings. Also, it is quite important that the input tensor is uniformly distributed, no missing values are allowed, and the input is numeric.

### 1.3 Factorization Machine

Until recently, de facto the silver bullet in the field of recommendation systems were factorization machines. They were firstly presented in ["Factorization Machines"](https://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf) by Rendel S. in 2010. It was inspired by and proposed as a substitution for traditional matrix factorization. This approach allows to learn interactions between user and movies, and at the same time involve user metadata for final prediction. Thus, this approach resolves well-known problem of the [cold-start](https://en.wikipedia.org/wiki/Cold_start_(recommender_systems)).

The developed model uses PyTorch embeddings to encode user, movie, age, occupation and gender. Therefore, the input tensor should consist of indices that can be refered to these embeddings. Also, it is quite important that the input tensor is uniformly distributed, no missing values are allowed, and the input is numeric.

### 1.4 Transformer

Finally, we decided to apply Behaviour Sequence Transformer (BST) architecture for our problem. The implementation is mainly inspired by manuscript ["Behaviour Sequence Transformer for E-commerce Recommendation in Alibaba"](https://arxiv.org/pdf/1905.06874.pdf). Long story short, authors expand MLP using sequence-to-sequence model with multi-head attention mechanism. Now the model not only takes into account metadata of user and movie, but also rating history. They claim to increase online Click-Through-Rate (CTR) gain by 7.57% compared to a control group using BST.

<img src="https://miro.medium.com/max/1400/1*ThXfG04qnukM6fqWLRb_JQ.jpeg" alt="drawing" width="700"/>

The developed model uses PyTorch embeddings to encode user, movie, age, occupation and gender. Therefore, the input tensor should consist of indices that can be refered to these embeddings. Also, it is quite important that the input tensor is uniformly distributed, no missing values are allowed, and the input is numeric. Moreover, the input sequence must have an upper limit, it order to feed the behaviour sequence in model.

## 2. Generate Test Design

In order to slightly relax our problem, let's reformulate it from regression to binary classification. Recall that in original dataset grades are from 1 to 5, thus new target is 1 if the grade above 3, and 0 - otherwise.

The dataset will be splitted in train, test, and validations subsets with size of approximately 70, 20, and 10% respectively. The data is splitted according to the timestamp to make sure that it inherits its historical order. Iteratively model is trained on train subset and tested on test subset (to check how model generalizes data, detect over-fitting, etc). Final performance of models is checked on validation subset.

Frankly speaking, the recommender system cannot be properly tested without external validation. Apparently, it seems that the best way is to design the A/B testing, and check how the new approach influenced certain metric, e.g. CTR. This project does not require deployment, so building infrastructure is out of the interest scope.

### 2.1 Data Preparation

In [4]:
movielens_dir = Path("./data/movie_lens/movie_lens_1m.csv")
joint_dir = Path("./data/augmented_movie_lens.csv")

In [5]:
movielens_df = pd.read_csv(movielens_dir)
joint_df = pd.read_csv(joint_dir)

In [6]:
def index(df, columns):
    # TODO: make assert
    _df = df.copy()
    offset = 0
    for column in columns:
        index_column = column + '_index'
        _df[index_column] = offset + _df[column].astype('category').cat.codes
        offset += len(_df[index_column].unique())
    return _df

columns = ['UserID', 'MovieID', 'Gender', 'Age', 'Occupation']
movielens_df_indexed = index(movielens_df, columns)

In [7]:
def split_dataset(df):
    _df = df.copy()
    train_df = _df[df.Timestamp <= 975e6]
    test_df = _df[(df.Timestamp > 975e6) & (df.Timestamp < 98e7)]
    val_df = _df[df.Timestamp >= 98e7]
    return train_df, test_df, val_df

In [8]:
train_movielens_df, test_movielens_df, val_movielens_df = split_dataset(movielens_df_indexed)
train_joint_df, test_joint_df, val_joint_df = split_dataset(joint_df)

In [9]:
assert len(movielens_df) == len(train_movielens_df) + len(test_movielens_df) + len(val_movielens_df)
assert len(joint_df) == len(train_joint_df) + len(test_joint_df) + len(val_joint_df)

In [10]:
movie_mapping = dict(movielens_df_indexed[['Title', 'MovieID_index']].drop_duplicates().values[:, ::-1])
gender_mapping = dict(movielens_df_indexed[['Gender', 'Gender_index']].drop_duplicates().values[:, ::-1])
age_mapping = dict(movielens_df_indexed[['Age', 'Age_index']].drop_duplicates().values[:, ::-1])
occupation_mapping = dict(movielens_df_indexed[['Occupation', 'Occupation_index']].drop_duplicates().values[:, ::-1])

In [11]:
features = ['UserID_index', 'MovieID_index', 'Age_index', 'Gender_index', 'Occupation_index']

In [12]:
binary_target = np.where(train_movielens_df.Rating.values < 4, 0, 1)
X = torch.LongTensor(train_movielens_df[features].values)
y = torch.LongTensor(binary_target).float()
trainset = data.TensorDataset(X, y)

binary_target = np.where(test_movielens_df.Rating.values < 4, 0, 1)
X = torch.LongTensor(test_movielens_df[features].values)
y = torch.LongTensor(binary_target).float()
testset = data.TensorDataset(X, y)

binary_target = np.where(val_movielens_df.Rating.values < 4, 0, 1)
X = torch.LongTensor(val_movielens_df[features].values)
y = torch.LongTensor(binary_target).float()
valset = data.TensorDataset(X, y)

In [13]:
trainloader = data.DataLoader(trainset, batch_size=1024, shuffle=True)
testloader = data.DataLoader(testset, batch_size=1024, shuffle=True)
valloader = data.DataLoader(valset, batch_size=1024, shuffle=True)

In [14]:
num_embeddings = movielens_df_indexed[features].values.max() + 1

### 2.2 Metric Implementation

The performance of model is defined in terms of percentage of suggested movies which the user rated 4 or 5 among suggested movies which user rated (further refered as accuracy).

In [15]:
def compute_accuracy(ratings, recommendation):
    """Compute accuracy of recommender system.

    Parameters
    ----------
    ratings : pandas.DataFrame
        dataframe with user's ratings
    
    recommendation : iterable 
        recommendations of a system

    Returns
    -------
    accuracy : float
        percentage of suggested movies which the user rated 4 or 5 
        among suggested movies which user rated
    """
    well_rated = ratings[ratings.Rating >= 4].MovieID_index
    all_rated = ratings.MovieID_index
    hits = len(list(set(well_rated) & set(recommendation)))
    intersection = len(list(set(all_rated) & set(recommendation)))
    if intersection == 0:
        return np.NaN
    accuracy = hits / intersection
    return accuracy

In [16]:
def compute_mean_accuracy(df, model):
    """Compute mean accuracy of inferenced model.

    Parameters
    ----------
    df : pandas.DataFrame
        validation dataset

    model : Object or torch.nn.Module
        instance of class implementing predict method
    
    Returns
    -------
    mean_accuracy : float
        mean accuracy of recommendations
    """
    accuracies = list()
    for user_id in df.UserID.unique():
        ratings = df[df.UserID == user_id]
        indices = model.predict(ratings)
        ratings = ratings[['MovieID_index', 'Rating']]
        accuracy = compute_accuracy(ratings, indices)
        accuracies.append(accuracy)
    accuracies = np.array(accuracies)
    accuracies = accuracies[~np.isnan(accuracies)]
    mean_accuracy = accuracies.mean()
    return mean_accuracy

## 3. Build Model

This section contains implementations for item popularity model, multilayer perceptron, factorization machine, and transformer.

### 3.1 Item Popularity Model

In [17]:
class ItemPopularityModel:
    """Implementation of Item Popularity model for recommender systems."""

    def __init__(self, top_k=50):
        """Constructor of Item Popularity model.

        Parameters
        ----------
        top_k : int
            number of items to recommend
        """
        self.top_k = top_k
        self.prediction = None

    def fit(self, df):
        """Training routine.

        Sorts movies by nubmer of ratings. Sets indices of top k movies in 
        attribute prediction.

        Parameters
        ----------
        df : pandas.DataFrame
            data to use for training
        """
        assert 'MovieID_index' in df and 'Rating' in df, '"MovieID_index" and "Rating" should be in column names'
        _df = df.copy()
        ratings_number = _df[['MovieID_index', 'Rating']].groupby(['MovieID_index']).count()
        sorted_titles = ratings_number.sort_values(by='Rating', ascending=False)
        sorted_titles.reset_index(inplace=True)
        self.prediction = sorted_titles
        
    def predict(self, df=None):
        """Inference model.

        Parameters
        ----------
        df : pandas.DataFrame
            data to use for training

        Returns
        -------
        indices : list
            indices of recommended movies
        """
        recommendation = self.prediction[:self.top_k]
        indices = recommendation['MovieID_index']
        return indices

In [18]:
item_popularity_model = ItemPopularityModel()
item_popularity_model.fit(train_movielens_df)

### 3.2 Multi-layer Perceptron

In [19]:
class MLP(nn.Module):
    """Implementation of Multilayer Perceptron with embeddings."""

    def __init__(self, n, layers=[600, 400, 200, 50]):
        """Constructor of class MLP
        
        Parameters
        ----------
        n : int
            number of entities to embed
        
        layers : list
            list of layers sizes
        """
        super().__init__()
        assert layers[0] % 5 == 0
        self.embeddings = nn.Embedding(n, layers[0] // 5)
        architecture = list()
        for idx in range(len(layers) - 1):
            architecture.append(nn.Linear(layers[idx], layers[idx + 1]))
            architecture.append(nn.ReLU())
        architecture.append(nn.Linear(layers[-1], 1))
        architecture.append(nn.Sigmoid())
        self.main = nn.Sequential(*architecture)
    
    def forward(self, x):
        embeded = self.embeddings(x)
        x = embeded.flatten(start_dim=1)
        x = self.main(x)
        return x

In [20]:
class MLPModel: 
    
    def __init__(self, n, trainloader, testloader,
                 lr=1e-3, weight_decay=1e-5, epochs=10, top_k=50, device="cuda"):
        self.n = n
        self.trainloader = trainloader
        self.testloader = testloader
        self.lr = lr
        self.weight_decay = weight_decay
        self.epochs = epochs
        self.device = device
        self.top_k = top_k
        
        self.model = MLP(n).to(device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr, weight_decay=weight_decay)
        self.scheduler = optim.lr_scheduler.MultiStepLR(self.optimizer, 
                                                        milestones=list(range(0, epochs + 1, 5))[1:], 
                                                        gamma=0.1)
        self.criterion = nn.BCELoss().to(device)
        
    def fit(self, verbose=False):
        "High-level outline of the training process."
        for epoch in range(self.epochs):
            train_loss = self._train_one_epoch()
            valid_loss = self._test()
            self.scheduler.step()
            if not verbose:
                print(f"Epoch #{epoch + 1} | Train loss: {train_loss:.4f} | Test loss: {valid_loss:.4f} |")
        if verbose:
            print(f"Final train loss: {train_loss:.4f}, test loss: {valid_loss:.4f}")
        return self.model
    
    def _train_one_epoch(self):
        "Train model for one epoch on train dataset."
        device = self.device
        train_loss = 0
        self.model.train()
        for x, y in self.trainloader:
            self.optimizer.zero_grad()
            y_hat = self.model(x.to(device)).flatten()
            loss = self.criterion(y_hat, y.to(device))
            train_loss += loss.item() * x.shape[0]
            loss.backward()
            self.optimizer.step()
        return train_loss / len(self.trainloader.dataset)
    
    def _test(self):
        "Test model on validation dataset."
        device = self.device
        test_loss = 0
        self.model.eval()
        for x, y in self.testloader:                    
            with torch.no_grad():
                y_hat = self.model(x.to(device)).flatten()
            loss = self.criterion(y_hat, y.to(device))
            test_loss += loss.item() * x.shape[0]
        return test_loss / len(self.testloader.dataset)
    
    def predict(self, x):
        user_id = int(x.UserID_index.unique())
        age = int(x.Age_index.unique())
        gender = int(x.Gender_index.unique())
        occupation = int(x.Occupation_index.unique())

        movies = movielens_df_indexed.drop_duplicates('MovieID_index').copy()

        rankings = []
        for movie in movies.MovieID_index.unique():
            ranking = self.model(torch.LongTensor([user_id, movie, age, gender, occupation]).unsqueeze(0).to(device))
            rankings.append(ranking.item())
        
        rankings = torch.FloatTensor(rankings)
        recommendations = [i for i in movies.iloc[rankings.argsort(descending=True).cpu()]['MovieID_index'].values][:self.top_k]
        return recommendations

In [21]:
mlp_model = MLPModel(num_embeddings, trainloader, testloader, device=device)
model = mlp_model.fit()

Epoch #1 | Train loss: 0.6033 | Test loss: 0.6121 |
Epoch #2 | Train loss: 0.5482 | Test loss: 0.5982 |
Epoch #3 | Train loss: 0.5350 | Test loss: 0.5908 |
Epoch #4 | Train loss: 0.5280 | Test loss: 0.5862 |
Epoch #5 | Train loss: 0.5218 | Test loss: 0.5855 |
Epoch #6 | Train loss: 0.5024 | Test loss: 0.5850 |
Epoch #7 | Train loss: 0.4974 | Test loss: 0.5852 |
Epoch #8 | Train loss: 0.4940 | Test loss: 0.5866 |
Epoch #9 | Train loss: 0.4909 | Test loss: 0.5888 |
Epoch #10 | Train loss: 0.4878 | Test loss: 0.5915 |


### 3.3 Factorization Machine

In [22]:
class FactorizationMachine(nn.Module):
    """Implementation of Factorization Machine acccording to "Factorization 
    Machines" by Rendle.
    https://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf
    """
    
    def __init__(self, n, k):
        """Constructor of FMM class.
        
        Parameters
        ----------
        n : int
            size of feature vector
        
        k : int
            size of embedding to use
        
        Returns
        -------
        None
        """
        super().__init__()
        self.w0 = nn.Parameter(torch.zeros(1))
        self.bias = nn.Embedding(n, 1)
        self.embeddings = nn.Embedding(n, k)
        self._init_trunc_normal()
    
    def _init_trunc_normal(self, mean=0., std=0.01):
        """Initialize weights via truncated normal function.
        
        Implemented according to "An Exploration of Word Embedding 
        Initialization in Deep-Learning Tasks" by Kocmi T. and Bojar O.
        https://arxiv.org/pdf/1711.09160.pdf
        
        Parameters
        ----------
        mean : float (default: 0.00)
            mean of normal distribution
        
        std : float (default: 0.01)
            standard deviation of normal distribution
        
        Returns
        -------
        None
        """
        with torch.no_grad(): 
            self.embeddings.weight.normal_().fmod_(2).mul_(std).add_(mean)
            self.bias.weight.normal_().fmod_(2).mul_(std).add_(mean)
    
    def forward(self, x):
        "Compute interactions using Lemma 3.1"
        bias = self.bias(x).squeeze().sum(1)
        embeded = self.embeddings(x)
        pow_of_sum = embeded.sum(dim=1).pow(2)
        sum_of_pow = embeded.pow(2).sum(dim=1)
        pairwise = (pow_of_sum - sum_of_pow).sum(1) * 0.5
        y = torch.sigmoid(self.w0 + bias + pairwise)
        return y

In [23]:
class FactorizationMachineModel: 
    
    def __init__(self, n, k, trainloader, testloader, 
                 lr=1e-3, weight_decay=1e-5, epochs=10, device="cuda", top_k=50):
        """Constructor of training routine.
        
        Parameters
        ----------
        n : int
            size of feature vector
        
        k : int
            size of embedding to use
        
        trainloader : torch.utils.data.DataLoader 
            iterator over training data
        
        testloader : torch.utils.data.DataLoader 
            iterator over test data
        
        lr : float
            learning rate for optimizer
        
        weight_decay : float
            regularization parameter for optimizer
        
        epochs : int
            number of training epochs
        
        device : str
            device to use for training
        
        Returns
        -------
        None
        """
        self.n = n
        self.k = k
        self.trainloader = trainloader
        self.testloader = testloader
        self.lr = lr
        self.weight_decay = weight_decay
        self.epochs = epochs
        self.device = device
        self.top_k = top_k
        
        self.model = FactorizationMachine(n, k).to(device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr, weight_decay=weight_decay)
        self.scheduler = optim.lr_scheduler.MultiStepLR(self.optimizer, 
                                                        milestones=list(range(0, epochs + 1, 5))[1:], 
                                                        gamma=0.1)
        self.criterion = nn.BCELoss().to(device)
        
    def fit(self, verbose=False):
        "High-level outline of the training process."
        for epoch in range(self.epochs):
            train_loss = self._train_one_epoch()
            valid_loss = self._test()
            self.scheduler.step()
            if not verbose:
                print(f"Epoch #{epoch + 1} | Train loss: {train_loss:.4f} | Test loss: {valid_loss:.4f} |")
        if verbose:
            print(f"Final train loss: {train_loss:.4f}, test loss: {valid_loss:.4f}")
        return self.model
    
    def _train_one_epoch(self):
        "Train model for one epoch on train dataset."
        device = self.device
        train_loss = 0
        self.model.train()
        for x, y in self.trainloader:
            self.optimizer.zero_grad()
            y_hat = self.model(x.to(device))
            loss = self.criterion(y_hat, y.to(device))
            train_loss += loss.item() * x.shape[0]
            loss.backward()
            self.optimizer.step()
        return train_loss / len(self.trainloader.dataset)
    
    def _test(self):
        "Test model on validation dataset."
        device = self.device
        test_loss = 0
        self.model.eval()
        for x, y in self.testloader:                    
            with torch.no_grad():
                y_hat = self.model(x.to(device))
            loss = self.criterion(y_hat, y.to(device))
            test_loss += loss.item() * x.shape[0]
        return test_loss / len(self.testloader.dataset)
    
    def predict(self, x):
        user_id = int(x.UserID_index.unique())
        age = int(x.Age_index.unique())
        gender = int(x.Gender_index.unique())
        occupation = int(x.Occupation_index.unique())

        movies = movielens_df_indexed.drop_duplicates('MovieID_index').copy()
        movie_embeddings = model.embeddings(torch.tensor(movies['MovieID_index'].values,device=device).long())
        movie_biases = model.bias(torch.tensor(movies['MovieID_index'].values,device=device).long())

        user_embedding = model.embeddings(torch.tensor(user_id,device=device))
        age_embedding = model.embeddings(torch.tensor(age,device=device))
        gender_embedding = model.embeddings(torch.tensor(gender,device=device))
        occupation_embedding = model.embeddings(torch.tensor(occupation,device=device))

        metadata_embedding = user_embedding + age_embedding + gender_embedding + occupation_embedding
        rankings = movie_biases.squeeze() + (metadata_embedding * movie_embeddings).sum(1)
        recommendations = [i for i in movies.iloc[rankings.argsort(descending=True).cpu()]['MovieID_index'].values][:self.top_k]
        return recommendations

In [24]:
factorization_machine = FactorizationMachineModel(num_embeddings, 120, trainloader, testloader, device=device)
model = factorization_machine.fit()

Epoch #1 | Train loss: 0.5659 | Test loss: 0.5850 |
Epoch #2 | Train loss: 0.5387 | Test loss: 0.5823 |
Epoch #3 | Train loss: 0.5326 | Test loss: 0.5818 |
Epoch #4 | Train loss: 0.5275 | Test loss: 0.5815 |
Epoch #5 | Train loss: 0.5224 | Test loss: 0.5812 |
Epoch #6 | Train loss: 0.5093 | Test loss: 0.5809 |
Epoch #7 | Train loss: 0.5072 | Test loss: 0.5809 |
Epoch #8 | Train loss: 0.5060 | Test loss: 0.5810 |
Epoch #9 | Train loss: 0.5050 | Test loss: 0.5810 |
Epoch #10 | Train loss: 0.5041 | Test loss: 0.5813 |


## 4. Assess Model

In [25]:
accuracy = compute_mean_accuracy(val_movielens_df, item_popularity_model)
print(f"Accuracy of Item Popularity model is {accuracy * 100:.2f}%")

Accuracy of Item Popularity model is 74.11%


In [28]:
accuracy = compute_mean_accuracy(val_movielens_df, mlp_model)
print(f"Accuracy of Multilayer Perceptron is {accuracy * 100:.2f}%")

Accuracy of Multilayer Perceptron is 85.41%


In [29]:
accuracy = compute_mean_accuracy(val_movielens_df, factorization_machine)
print(f"Accuracy of Factorization Machine is {accuracy * 100:.2f}%")

Accuracy of Factorization Machine is 89.31%
